# Tokenizador BERT
Vamos a ver cómo funciona el tokenizado WordPiece que utiliza el modelo BERT. Usamos la librería `transformers`. Se puede instalar en Anaconda con  
```
conda install -c huggingface transformers
```


In [ ]:
#instalamos la librería (Google Colab)
!pip install transformers

In [ ]:
from transformers import AutoTokenizer

Exploramos el vocabulario del modelo `bert-base-cased`(inglés)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
tokenizer.vocab_size

In [ ]:
import numpy as np

np.random.choice(list(tokenizer.vocab.keys()), 10)

El modelo BERT utiliza un tokenizado subword de tipo *WordPiece*:


In [ ]:
output = tokenizer.tokenize("the BERT tokenizer was created with a WordPiece model")
print(output)

In [ ]:
len(output)

Para utilizar un documento tokenizado en BERT tenemos que convertirlo en tokens con el método `tokenizer`

In [ ]:
output = tokenizer("the BERT tokenizer was created with a WordPiece model")


In [ ]:
output

In [ ]:
output.keys()

In [ ]:
len(output.input_ids)

El tokenizador asigna cada token a su `token_id` correspondiente en el vocabulario

In [ ]:
import pandas as pd

tokens = map(lambda t: {'token_id': t,
                        'token': tokenizer.convert_ids_to_tokens(t)},
             output.input_ids)

pd.DataFrame(tokens)

El modelo BERT añade unos tokens especiales al inicio y al final de cada documento:

In [ ]:
#tokens especiales
tokenizer.convert_ids_to_tokens([0, 100, 101, 102, 103])

Tokenizado de pares de frases (para aplicaciones con dos documentos de entrada, como NLI o QA)

In [ ]:

output = tokenizer([["this is the first sentence",
    "this is the second one"]])
output

In [ ]:
print(tokenizer.decode(output['input_ids'][0]))

### Tokenizado de un corpus de documentos
Cuando se tokeniza más de un documento, es necesario ajustar la longitud de todos al mismo número de tokens (con *padding/truncating*)

In [ ]:
text = [
    "This movie was great!",
    "I hated this move, waste of time!",
    "Epic?"
]

encoded = tokenizer(text, padding=True, add_special_tokens=True)

print("**Input IDs**")
for a in encoded.input_ids:
    print(a)

print("**Attention Mask**")
for a in encoded.attention_mask:
    print(a)

## Análisis del vocabulario
El vocabulario contiene palabras completas y partes de palabra (tokens que comienzan por '##')

In [ ]:
one_chars = []
one_chars_hashes = []

# recorremos todos los tokens del vocabulario
for token in tokenizer.vocab.keys():
    
    # guardamos los tokens de un solo caracter
    if len(token) == 1:
        one_chars.append(token)
    
    # guardamos los tokens de un solo caracter precedidos de ##    
    elif len(token) == 3 and token[0:2] == '##':
        one_chars_hashes.append(token)

In [ ]:
print(one_chars)

In [ ]:
print(one_chars_hashes)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

sns.set(style='darkgrid')

# Increase the plot size and font size.
sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (10,5)

# Measure the length of every token in the vocab.
token_lengths = [len(token) for token in tokenizer.vocab.keys()]

# Plot the number of tokens of each length.
sns.countplot(x=token_lengths)
plt.title('Longitud de los tokens del vocabulario')
plt.xlabel('Longitud del token')
plt.ylabel('Nº de Tokens')

print('Longitud de token máxima:', max(token_lengths))

Ahora analizamos sólo las subpalabras (tokens que empiezan por `'##'` y no son caracteres únicos)

In [ ]:
num_subwords = 0

subword_lengths = []

# For each token in the vocabulary...
for token in tokenizer.vocab.keys():
    
    # If it's a subword...
    if len(token) >= 2 and token[0:2] == '##':
        
        # Tally all subwords
        num_subwords += 1

        # Measure the sub word length (without the hashes)
        length = len(token) - 2

        # Record the lengths.        
        subword_lengths.append(length)

sns.countplot(x=subword_lengths)
plt.title('Longitud de los subwords del vocabulario (sin "##")')
plt.xlabel('Longitud subtoken')
plt.ylabel('Nº de subwords')

print(f"Número de subwords: {len(subword_lengths)}")


## Tokenizado en otros idiomas
El modelo BERT en inglés no conoce las palabras del vocabulario español
### Ejercicio
Convierte en tokens la frase '*El modelo BERT en inglés no conoce el vocabulario español*' con el modelo BERT previamente cargado y con el modelo multilingüe `nlptown/bert-base-multilingual-uncased-sentiment`. Compara las diferencias.

In [ ]:
#Modelo BERT inglés


En cambio el mismo texto tokenizado con un modelo BERT multilingüe sí conoce las palabras:

In [ ]:
#Modelo BERT multilingüe